In [ ]:
#importing necessary libraries
from keras.models import load_model
import cv2
import numpy as np
import tkinter
from tkinter import messagebox
import smtplib

#initialize tkinter
root=tkinter.Tk()
root.withdraw()

#load trained deep learning model
model= load_model('face_mask_detection_alert_system.h5')

#Classifier to detect face
face_det_classifier=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


#Capture Video
vid_source=cv2.VideoCapture(0)

text_dict={0:'Mask On' , 1: 'No Mask'}
rect_color_dict={0:(0,255,0),1:(0,0,255)}

SUBJECT="Subject"
TEXT="One Visitor Violated Face Mask Policy"

#while loop to continiously detect camera feed

while(True):
    ret,img=vid_source.read()
    greyscale_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_det_classifier.detectMultiScale(greyscale_img,1.3,5)
    
    for(x,y,w,h) in faces:
        face_img=greyscale_img[y:y+w,x:x+w]
        
        resized_img=cv2.resize(face_img,(112,112))
        normalized_img= resized_img/255.0
        reshaped_img=np.reshape(normalized_img,(1,112,112,1))
        result=model.predict(reshaped_img)

        label=np.argmax(result, axis=1)[0]

        cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
        cv2.putText(img,text_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)

        # if label==1 then it means not wearing a mask and 0 means wearing mask
        if(label==1):
            #throw a message warning to tell user to wear a  mask if not wearing one. this will stay open and no access will be given till he/she wears a mask
            messagebox.showwarning("Warning","Access Denied. Please Wear A Mask")

            #send an email to the admin if access denied/user not wearing face mask
            message = "Subject:{}\n\n{}".format(SUBJECT,TEXT)
            mail=smtplib.SMTP('smtp.gmail.com',587)
            mail.ehlo()
            mail.starttls()
            mail.login('seth.satvik05@gmail.com','zombies12345')
            mail.sendmail('seth.satvik05@gmail.com','seth.satvik05@gmail.com',message)
        else:
            pass
            break
    cv2.imshow('Live Feed',img)
    key=cv2.waitKey(1)

    if(key==27):
        break